In [ ]:
!nvidia-smi

Fri Jul 23 10:43:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install datasets transformers rouge-score nltk py7zr
! pip install wandb -qq

     |████████████████████████████████| 542 kB 8.5 MB/s 
     |████████████████████████████████| 2.6 MB 69.1 MB/s 
     |████████████████████████████████| 65 kB 2.3 MB/s 
     |████████████████████████████████| 76 kB 6.2 MB/s 
     |████████████████████████████████| 118 kB 75.7 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 243 kB 59.2 MB/s 
     |████████████████████████████████| 895 kB 58.9 MB/s 
     |████████████████████████████████| 3.3 MB 51.8 MB/s 
     |████████████████████████████████| 636 kB 49.2 MB/s 
     |████████████████████████████████| 121 kB 65.7 MB/s 
     |████████████████████████████████| 1.9 MB 55.5 MB/s 
     |████████████████████████████████| 2.2 MB 44.8 MB/s 
     |████████████████████████████████| 357 kB 56.9 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existin

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: nid989 (use `wandb login --relogin` to force relogin)


True

In [ ]:
%env WANDB_PROJECT=bart_large_xsum_samsum

env: WANDB_PROJECT=bart_large_xsum_samsum


In [ ]:
model_checkpoint = "facebook/bart-large-xsum"

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("samsum")
metric = load_metric("rouge")

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/770 [00:00<?, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "test-dialogue-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",  # enable logging to W&B
    run_name="bart-large-xsum-samsum"  # name of the W&B run (optional)
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, dialogue, id.
***** Running training *****
  Num examples = 14732
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 5523
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1310: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.384800,1.447523,53.373100,28.897700,44.133000,48.806400,25.792200
1,1.047500,1.478893,54.286800,29.193600,44.496500,49.723900,29.298300
2,0.856500,1.488593,54.532000,29.692300,45.149800,50.202500,30.198000


Saving model checkpoint to test-dialogue-summarization/checkpoint-500
Configuration saved in test-dialogue-summarization/checkpoint-500/config.json
Model weights saved in test-dialogue-summarization/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-dialogue-summarization/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-dialogue-summarization/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-dialogue-summarization/checkpoint-1000
Configuration saved in test-dialogue-summarization/checkpoint-1000/config.json
Model weights saved in test-dialogue-summarization/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-dialogue-summarization/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-dialogue-summarization/checkpoint-1000/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1310: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; 

TrainOutput(global_step=5523, training_loss=1.116441732900807, metrics={'train_runtime': 5549.3066, 'train_samples_per_second': 7.964, 'train_steps_per_second': 0.995, 'total_flos': 3.003043192720589e+16, 'train_loss': 1.116441732900807, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, dialogue, id.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 4


{'epoch': 3.0,
 'eval_gen_len': 30.198,
 'eval_loss': 1.4885928630828857,
 'eval_rouge1': 54.532,
 'eval_rouge2': 29.6923,
 'eval_rougeL': 45.1498,
 'eval_rougeLsum': 50.2025,
 'eval_runtime': 240.962,
 'eval_samples_per_second': 3.395,
 'eval_steps_per_second': 0.851}

In [ ]:
wandb.finish()

train/loss,0.8565
train/learning_rate,0.0
train/epoch,3.0
train/global_step,5523
_runtime,5855
_timestamp,1627043108
_step,15
eval/loss,1.48859
eval/rouge1,54.532
eval/rouge2,29.6923
eval/rougeL,45.1498


train/loss,█▇▆▅▃▃▃▂▁▁▁
train/learning_rate,█▇▇▆▅▅▄▃▂▂▁
train/epoch,▁▂▂▃▃▄▄▅▅▆▇▇████
train/global_step,▁▂▂▃▃▄▄▅▅▆▇▇████
_runtime,▁▂▂▃▃▄▄▅▅▅▆▇▇███
_timestamp,▁▂▂▃▃▄▄▅▅▅▆▇▇███
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
eval/loss,▁▆██
eval/rouge1,▁▇██
eval/rouge2,▁▄██
eval/rougeL,▁▄██


[Uploaded the model](https://huggingface.co/transformers/model_sharing.html) to the [🤗 Model Hub](https://huggingface.co/models). You can use it to generate results as shown below.

In [1]:
!pip install --quiet transformers

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.3 MB/s 
     |████████████████████████████████| 596 kB 44.6 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 


In [2]:
from transformers import pipeline

summarizer = pipeline("summarization", model="lidiya/bart-large-xsum-samsum")

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
import re
import json
from tqdm.notebook import tqdm
import os
import datetime 

In [7]:
# list of regular expression to be forged in transcripts
reg_ex = {
    r"( *)\<(.*?)\>": '',
    r"\n": '',
    r"(  +)": ' ',
    r"\-": '',
    r"(,+)": ',',
    r"( *)(-+)": '',
    r"\[": '',
    r"\]": '',   
}

In [8]:
# utility methods 

# open transcript using given file_path
def open_transcript(file_path):
  document = open(file_path, "r").readlines()
  return document

# remove punctuations and special tokens
def rem_ntok(reg_ex, text):
  for key, val in reg_ex.items():
    text = re.sub(key, val, text)
  return text

# remove extra space from the text
# remove \n and concat utterance which do not start with (PERSON.*?)
def add_colon(sentence):
  eidx = re.search(r'\(PERSON(.*?)\)', sentence).end()
  if sentence[eidx] == ':':
    return sentence
  else:
    return sentence[:eidx] + ':' + sentence[eidx:]

# process roles list and remove "( and )"
def process_roles(role):
  regex = {
      r"\(": '',
      r"\)": ''
  }
  for key, value in regex.items():
    role = re.sub(key, '', role)
  return role

# remove special tokens from the processed list of roles and utterances
def remove_special_tokens(utterance):
  regex = [r'^\.\',', r'^\.\'', r'^\',', r'^,', r'^\'', r'^\.', r'^, ,', r'^\?']
  for exp in regex:
    utterance = re.sub(exp, '', utterance)
  return utterance

# retur max_lenght of list of sentences
def max_length(text_list):
  length = [len(text.split(' ')) for text in text_list]
  return max(length)

# remove short utterances precisely "4"
def preprocess_utterance(sequence):
   return_seq = [sentences for sentences in sequence if len(sentences) > 4]
   return return_seq

# insert extra roles based on generated sentences
def insert_to_roles(roles, len, idx, role, con_index):
  idx = idx + con_index
  for i in range(len):
    roles.insert(idx, role)
  return roles

# text insertion in utterance list at a particular position.
def insert_text(utterances, sequences, idx, con_index):
  idx = idx + con_index
  for text in sequences[::-1]:
    utterances.insert(idx, text)
  return utterances

# check if folder contains transcripts
def check_for_transcript(file_list):
  for file_ in file_list:
    result = re.findall("transcript", file_)
    if len(result) == 1:
      return file_
    else:
      pass
  return ValueError("File not found!")

# convert to json files and save
def to_JSON_batch(processed_dict, file_path):
  with open(file_path, "w") as file_handle:
    json.dump(processed_dict, file_handle)

def to_JSON_single(processed_dict, file_name, file_path):
  out_dict = {file_name: processed_dict}
  with open(file_path, "w") as file_handle:
    json.dump(out_dict, file_handle)

In [9]:
# remove newline character
def preprocess_transcripts(document):
  transcript = []
  for line in document:
    if line == "\n":
      continue
    transcript.append(line.replace("\n", "") + " ")
  return transcript

# iterate over transcript and segmentation
def parse_transcript(reg_ex, transcript):
  # updated list of transcript's text
  updateList = []
  for text in transcript:
    updateList.append(rem_ntok(
        reg_ex = reg_ex,
        text = text
    ))
  # create list of utterances
  utteranceList = []
  person_regex = [r'\(PERSON(.*)\)']
  for text in updateList:
    result = re.findall(person_regex[0], text)
    if len(result) == 1:
      utteranceList.append(add_colon(text))
    else:
      try:
        prev_text = utteranceList[-1]
        utteranceList[-1] = prev_text + text.strip() + " "
      except Exception as e:
        pass
  return utteranceList

# bifurcate transcripts into roles and utterances. 
def split_transcripts(processed_transcript):
  roles, utterances, temp_roles = [], [], []
  for text in processed_transcript:
    temp = text.split(':')
    tune = remove_special_tokens(temp[1].strip()).strip()
    tune = remove_special_tokens(tune.strip()).strip()
    tune = remove_special_tokens(tune.strip()).strip()
    if tune is not '' and len(tune) > 2:
      utterances.append(tune)
      temp_roles.append(temp[0])
  for role in temp_roles:
    roles.append(process_roles(role))
  return roles, utterances

In [10]:
# shortning and splitting utterance sentence and assign roles!

def post_process(roles, utterances):
  mappings = {
      "idx": [],
      "utterances": [],
      "roles": []
  }
  for idx, utterance in enumerate(utterances):
    word_list = [sentence.strip() for sentence in utterance.split(' ')]
    sentence_list = [sentence.strip() for sentence in utterance.split('.')]
    # check if length of word list is greater than 150
    if len(word_list) > 150:
      sequence = []
      temp = ""
      for sentence in sentence_list:
        temp = f'{temp} {sentence}.'
        # if word limit exceeded than create a new sentence
        if len(temp.split(' ')) > 150:
          sequence.append(temp.strip())
          temp = ''
      sequence.append(temp.strip())
      # delete the sentence present in original list
      del utterances[idx]
      # preprocess and striping and removing small sentence less than 3
      sequence = preprocess_utterance(sequence)
      len_roles = len(sequence)
      # retrieve corresponding role from the roles list
      role = roles[idx]
      # delete the role present in original list
      del roles[idx]
      # mapping index, roles and utterances to mapping dictionary
      mappings["idx"].append(idx)
      mappings["utterances"].append(sequence)
      mappings["roles"].append(role)
  # Applying modifications
  con_index = 0
  for idx, index in enumerate(mappings['idx']):
    sequence = mappings['utterances'][idx]
    len_utterances = len(sequence)
    utterances = insert_text(utterances, sequence, index, con_index)
    roles = insert_to_roles(roles, len_utterances, index, mappings['roles'][idx], con_index)
    # Reflecting to the position of insertion
    # print(f'Inserted @ {index + con_index}')
    con_index = con_index + len_utterances  

  # New length after insertion
  # print(f'Length of lists after insertion {len(roles), len(utterances)}')

  # Applying changes to main dictionary
  return roles, utterances

In [11]:
# process single file
def process_single(path_to_file):
  document = open_transcript(path_to_file)
  transcripts = preprocess_transcripts(document)
  transcripts = parse_transcript(reg_ex, transcripts)
  roles, utterances = split_transcripts(transcripts)
  roles, utterances = post_process(roles, utterances)
  trans_dict = {
      "roles": roles,
      "utterances": utterances
  }
  return trans_dict

# process batch files/ dataset
def batch_process(path_to_folder):
  folders = os.listdir(path_to_folder)
  main_dict = dict()
  for folder in tqdm(folders):
    try:
      root_folder = path_to_folder + f"{folder}/"
      files = os.listdir(root_folder)
      filename = check_for_transcript(files)
      trans_dict = process_single(f"{root_folder}{filename}")
      main_dict[folder] = trans_dict
    except Exception as e:
      pass
  return main_dict

In [31]:
# pre-segmentation utility methods
# generate samsum dataset favourable sentences 
def generate_dialogues(roles, utterances):
  sentences = zip(roles, utterances)
  dialogues = [f'{role}: {utterance}' for role, utterance in sentences]
  return dialogues

# partition document for better processing.
def doc_partitioning(document, max_characters=1700):
  processed_dict = dict()
  processed_dict['part_0'] = ''
  identity_generator = 'part_'
  temp = ''
  count = 0
  for sentence in document:
    key = f'{identity_generator}{count}'
    temp = temp + sentence
    if len(temp) > max_characters:
      temp = ''
      count = count + 1
      key = f'{identity_generator}{count}'
      processed_dict[key] = ''
    processed_dict[key] = processed_dict[key] +'\n'+  sentence
  return processed_dict

In [76]:
# summarization and minute generation.
def apply_summarizer(processed_dict):
  output = []
  for key in tqdm(processed_dict.keys(), total=len(processed_dict.keys())):
    result = summarizer(processed_dict[key])
    output.append(result[0]['summary_text'])
  return output

# split summarized paragraph into separate sentences.
def split_Sentences(summarizer_output):
  result = []
  for text in summarizer_output:
    sentences = text.split('.')
    for sentence in sentences:
      sentence = sentence.strip()
      if sentence != '':
        result.append(sentence)
  return result

# get current date.
def return_date():
  return f'Date: {datetime.datetime.now().strftime("%Y-%m-%d")}'

# prepare main-body of the meeting minutes.
def main_body(output):
  body = []
  body_header = 'SUMMARY- \n'
  for sentence in output:
    if len(sentence.split(' ')) > 3:
      body.append(f'-{sentence}')
  body_content = '\n'.join(body)  
  body_Block = f'{body_header}{body_content}'
  return body_Block

# retrieve meeting participants
def generate_person_list(output):
  reg_ex = [r'PERSON[0-9]{1,2}']
  person_list = []
  for sentence in output:
    result = re.search(reg_ex[0], sentence)
    try:
      person_list.append(result.group())
    except:
      pass
  return list(set(person_list))

# generate attendee string
def generate_attendees(person_list):
  attendee_header = 'ATTENDEES: '
  attendee_content = attendee_header + ', '.join(person_list)
  return attendee_content

# generate meeting minute
def prepare_document(attendee_str, body, annotator = 'Nidhir'):
  Date_ = return_date()
  Document = f'{Date_}\n{attendee_str}\n\n\n{body}\n\nMinuted by: {annotator}'
  return Document

In [68]:
# sample processing for single transcript
path_to_file = "/content/drive/MyDrive/automin-2021-confindential-data-main/task-A-elitr-minuting-corpus-en/test_II/meeting_en_test_025/transcript_MAN_annot17.deidentified.txt"
trans_dict = process_single(path_to_file)
trans_dict.keys()

# merge roles and utterances in conversational format provided by the samsum dataset.
meeting_conv = generate_dialogues(trans_dict["roles"], trans_dict["utterances"])

# pratitioned document
processed_dict = doc_partitioning(meeting_conv, max_characters=2000)

# apply summarizer to processed transcript
output = apply_summarizer(processed_dict)
output = split_Sentences(output)

  0%|          | 0/22 [00:00<?, ?it/s]

In [77]:
# generate different segments of the processed meeting
person_List = generate_person_list(output)
attendees = generate_attendees(person_List) 
main_body = main_body(output)

# Assemle parts and preparing final minute:
DOCUMENT = prepare_document(attendees, main_body)
print(DOCUMENT)

Date: 2021-11-12
ATTENDEES: PERSON6, PERSON1, PERSON3, PERSON4, PERSON2, PERSON5


SUMMARY- 
-PERSON1 was in a call until the very last minute and his machine crashed, so he had to restart it
-Now he can hear people again
-His mother is writing the question for the German subti subti subtitle user study and she has some progress
-PERSON4 started to write the deliverables and asked PERSON5 to do the transcripts of by German ASR f for the German transcriber
-PERSON5 has just added the transcripts for the first 30 parts and for the next ten parts, but for some folders the audio format is wrong
-PERSON1 and PERSON5 want to implement natural shortening in their models
-PROJECT4 is not ready for speech translation
-PERSON1, PERSON5, PERSON6 and PERSON3 are looking for a candidate for the PROJECT4 project
-PERSON1, PERSON6 and PERSON3 are going to have a dry run of some past similar talks to (PROJECT2) (docs) tomorrow afternoon
-PERSON3 got access to the platform yesterday
-He will complete t

In [84]:
path_to_file = "/content/drive/MyDrive/automin-2021-confindential-data-main/task-A-elitr-minuting-corpus-en/test_II/meeting_en_test_025/transcript_MAN_annot17.deidentified.txt"
transcript = open(path_to_file, "r").readlines()
transcript = preprocess_transcripts(transcript)
print("Transcript: \n")
for sentence in transcript:
  print(sentence)

Transcript: 

﻿(PERSON1) Hi, hello. 
Can you hear me? 
(PERSON2) Yes. 
(PERSON1) I don't hear anyone. 
I have to reconnect again. 
Or maybe can can someone else, give it a try as well. 
Because uh. 
(PERSON2) I can hear you. 
(PERSON1) [PERSON5], can you say something as well? 
Because wi- with [PERSON2], I remember that uh [PERSON2] also had uh some microphone issues at times. 
(PERSON2) And and you can hear- you cannot hear me or? 
(PERSON1) Yeah, so uh so I'll try to reconnect uh I'll uh yeah. 
Uh. 
So [PERSON3].  
Hello, can you say something? 
(PERSON3) Uh yeah, hi. 
(PERSON1) <other_yawn> 
Yeah, I.  
Uh. 
Uh I will have to reconnect uh- 
Making [PERSON3] the host. 
Uh.  
For now. 
<other_yawn> 
Yeah, so I I'm I'll make [PERSON3] the host for now. 
And I r- -re- reconnect. 
And leave meeting. 
(PERSON2) Oh, okay. 
I hear you, by the way. 
Hello.  
(PERSON1) Yeah, yeah. 
Now I can hear it. 
So for some reason I have always has to connect a few times uh until Zoom starts sending als